<a href="https://colab.research.google.com/github/groovu/dh-eucr/blob/master/dh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

url = "https://api.pushshift.io/reddit/search/submission"

def crawl_page(subreddit: str, last_page = None):
  """Crawl a page of results from a given subreddit.

  :param subreddit: The subreddit to crawl.
  :param last_page: The last downloaded page.

  :return: A page or results.
  """
  params = {"subreddit": subreddit, "size": 500, "sort": "desc", "sort_type": "created_utc"}
  if last_page is not None:
    if len(last_page) > 0:
      # resume from where we left at the last page
      params["before"] = last_page[-1]["created_utc"]
    else:
      # the last page was empty, we are past the last page
      return []
  results = requests.get(url, params)
  if not results.ok:
    # something wrong happened
    raise Exception("Server returned status code {}".format(results.status_code))
  return results.json()["data"]

In [ ]:
sol = crawl_page('berkeley')

In [ ]:
print(len(sol))

500


In [ ]:
import time

def crawl_subreddit(subreddit, max_submissions = 55084):
  """
  Crawl submissions from a subreddit.

  :param subreddit: The subreddit to crawl.
  :param max_submissions: The maximum number of submissions to download.

  :return: A list of submissions.
  """
  submissions = []
  last_page = None
  while last_page != [] and len(submissions) < max_submissions:
    last_page = crawl_page(subreddit, last_page)
    submissions += last_page
    time.sleep(3)
  return submissions[:max_submissions]

In [ ]:
lastest_submissions = crawl_subreddit("berkeley")


In [ ]:
print(len(lastest_submissions))
print(type(lastest_submissions))

print(type(lastest_submissions[1]))
print(lastest_submissions[1])
print(lastest_submissions[1].get('all_awardings'))
print(lastest_submissions[1].values())

55036
<class 'list'>
<class 'dict'>
{'all_awardings': [], 'allow_live_comments': False, 'author': 'bayleafcandle', 'author_flair_css_class': None, 'author_flair_richtext': [], 'author_flair_text': None, 'author_flair_type': 'text', 'author_fullname': 't2_h0etji', 'author_patreon_flair': False, 'author_premium': False, 'awarders': [], 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1592260263, 'domain': 'self.berkeley', 'full_link': 'https://www.reddit.com/r/berkeley/comments/h9qftg/has_anyone_taken_a_class_with_malone_or_ortiz/', 'gildings': {}, 'id': 'h9qftg', 'is_crosspostable': True, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_robot_indexable': True, 'is_self': True, 'is_video': False, 'link_flair_background_color': '', 'link_flair_richtext': [], 'link_flair_text_color': 'dark', 'link_flair_type': 'text', 'locked': False, 'media_only': False, 'no_follow': True, 'num_comments': 0, 'num_crossposts': 0, 'over_18': False, 'parent_whi

In [ ]:
def cleanup(list):
    cleaned = []
    i = 0;
    for d in list:
      cleaned += [[d.get('created'), (d.get('title'), d.get('self_text')]]
      print(d)
    
    return cleaned

       
    

In [ ]:
test = lastest_submissions[:10]

In [ ]:
print(test)
print(len(test))
clean = cleanup(test)
print(clean[1])

[{'all_awardings': [], 'allow_live_comments': False, 'author': 'asilvermourning', 'author_flair_css_class': None, 'author_flair_richtext': [], 'author_flair_text': None, 'author_flair_type': 'text', 'author_fullname': 't2_630ru2s', 'author_patreon_flair': False, 'author_premium': False, 'awarders': [], 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1592262616, 'domain': 'self.berkeley', 'full_link': 'https://www.reddit.com/r/berkeley/comments/h9r6bu/sigma_phi_thorsen_house/', 'gildings': {}, 'id': 'h9r6bu', 'is_crosspostable': True, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_robot_indexable': True, 'is_self': True, 'is_video': False, 'link_flair_background_color': '', 'link_flair_richtext': [], 'link_flair_text_color': 'dark', 'link_flair_type': 'text', 'locked': False, 'media_only': False, 'no_follow': True, 'num_comments': 0, 'num_crossposts': 0, 'over_18': False, 'parent_whitelist_status': 'all_ads', 'permalink': '/r/berkeley/c

In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  json.dump(test, f)

files.download('example.txt')

In [ ]:
def give_me_intervals(start_at, number_of_days_per_interval = 3):
    
    end_at = math.ceil(datetime.utcnow().timestamp())
        
    ## 1 day = 86400,
    period = (86400 * number_of_days_per_interval)

    end = start_at + period
    yield (int(start_at), int(end))

    padding = 1
    while end <= end_at:
        start_at = end + padding
        end = (start_at - padding) + period
        yield int(start_at), int(end)

def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)

    current_tries = 1
    while current_tries < max_retries:
        try:
            response = fire_away(uri)
            return response
        except:
            time.sleep(.150)
            current_tries += 1

    return fire_away(uri)
 
def pull_posts_for(subreddit, start_at, end_at):
    
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts( \
        make_request( \
            URI_TEMPLATE.format(subreddit, start_at, end_at, SIZE))['data'])

    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - 10
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format(subreddit, new_start_at, end_at, SIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)

    return post_collections

In [ ]:
subreddit = 'berkeley'

end_at = math.ceil(datetime.utcnow().timestamp())
start_at = math.floor((datetime.utcnow() - timedelta(days=10)).timestamp())

posts = pull_posts_for(subreddit, start_at, end_at)

print('found:', len(posts))
print('unique:', len(np.unique([ post['id'] for post in posts ])))

found: 464
unique: 464


In [ ]:
## assert no dups,
sorted_by_occurence = sorted([ (k, len(list(g))) for k,g in itertools.groupby(posts, lambda x: x['id']) ], key=lambda x: x[1], reverse=True)
sorted_by_occurence[:10]

[('gxfgun', 1),
 ('gxfjch', 1),
 ('gxft9j', 1),
 ('gxgctm', 1),
 ('gxgg6p', 1),
 ('gxgl1r', 1),
 ('gxgnqy', 1),
 ('gxh5rz', 1),
 ('gxhjat', 1),
 ('gxhxwm', 1)]

In [ ]:

## quick look at the posts,
posts[:10]

[{'created_utc': 1591399178, 'id': 'gxfgun', 'prefix': 't4_'},
 {'created_utc': 1591399418, 'id': 'gxfjch', 'prefix': 't4_'},
 {'created_utc': 1591400375, 'id': 'gxft9j', 'prefix': 't4_'},
 {'created_utc': 1591402276, 'id': 'gxgctm', 'prefix': 't4_'},
 {'created_utc': 1591402614, 'id': 'gxgg6p', 'prefix': 't4_'},
 {'created_utc': 1591403099, 'id': 'gxgl1r', 'prefix': 't4_'},
 {'created_utc': 1591403372, 'id': 'gxgnqy', 'prefix': 't4_'},
 {'created_utc': 1591405214, 'id': 'gxh5rz', 'prefix': 't4_'},
 {'created_utc': 1591406618, 'id': 'gxhjat', 'prefix': 't4_'},
 {'created_utc': 1591408148, 'id': 'gxhxwm', 'prefix': 't4_'}]